In [2]:
import edge_tts
import asyncio
import util   
import config
import json
import sys
from IPython.display import Audio
from rvc.rvc_speakers_vc_func import RVCSpeakers


2023-08-12 21:42:22 | INFO | faiss.loader | Loading faiss with AVX2 support.
2023-08-12 21:42:22 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


In [4]:

abs_path = sys.path[0] 

hubert_model_path=abs_path+'/rvc/model/hubert_base.pt'
rvc_config_file = abs_path+'/rvc/multi_config.json'

rvc = RVCSpeakers(hubert_model_path=hubert_model_path, rvc_config_file=rvc_config_file)

2023-08-12 21:42:47 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers
2023-08-12 21:42:47 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-12 21:42:47 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout

Loading model: yiqing
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: lulu
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: maomao
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Models loaded: 6


In [5]:


async def edge_tts_vc_func(
        rvc: RVCSpeakers,input_text, model_index, tts_speaker, pitch_adjust, f0_method, 
        feat_ratio, filter_radius, rms_mix_rate, resample_option,
        rate: str, volume: str
):
    if input_text is None:
        return (None, 'Please provide TTS text.')

    if tts_speaker is None:
        return (None, 'Please select TTS speaker.')

    if model_index is None:
        return (None, 'Please select a model.')

    speaker = rvc.tts_speakers_list[tts_speaker]['ShortName']
    (tts_np, tts_sr) = await util.call_edge_tts_config(speaker_name=speaker, text=input_text,rate=rate, volume=volume)
    return rvc.vc_func(
        (tts_sr, tts_np),
        model_index,
        pitch_adjust,
        f0_method,
        feat_ratio,
        filter_radius,
        rms_mix_rate,
        resample_option
    )


In [6]:

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc.loaded_models
]
loaded_models

['Bilibili - 一清清清',
 'ALL - Bob Sponge',
 'ALL - Ariana Grande',
 'ALL - Stefanie Sun',
 '露露 - 露露',
 '毛毛 - 毛毛']

In [7]:

model_index = loaded_models.index('露露 - 露露')
model_index

4

In [8]:

tts_speaker = [
    '%s (%s)' % (
        s['FriendlyName'],
        s['Gender'], 
    )
    for s in rvc.tts_speakers_list
] 

In [9]:
"""

 'Microsoft HiuGaai Online (Natural) - Chinese (Cantonese Traditional) (Female)',
 'Microsoft HiuMaan Online (Natural) - Chinese (Hong Kong) (Female)',
 'Microsoft WanLung Online (Natural) - Chinese (Hong Kong) (Male)',
 'Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)',
 'Microsoft Xiaoyi Online (Natural) - Chinese (Mainland) (Female)',
 'Microsoft Yunjian Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunxi Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunxia Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunyang Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Xiaobei Online (Natural) - Chinese (Northeastern Mandarin) (Female)',
 'Microsoft HsiaoChen Online (Natural) - Chinese (Taiwan) (Female)',
 'Microsoft YunJhe Online (Natural) - Chinese (Taiwan) (Male)',
 'Microsoft HsiaoYu Online (Natural) - Chinese (Taiwanese Mandarin) (Female)',
 'Microsoft Xiaoni Online (Natural) - Chinese (Zhongyuan Mandarin Shaanxi) (Female)',
"""
tts_speaker_index = tts_speaker.index('Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)')
tts_speaker_index

55

In [11]:
value="封建五爵，井田刑狱"
# 变调
pitch_adjust=0
f0_method =['pm', 'harvest']

feat_ratio=0.6
filter_radius=3
rms_mix_rate=1
resample_rate=0

rate='-20%'
volume='-50%' 
out_sr, output_audio = await edge_tts_vc_func(rvc,value,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                   rate=rate, volume=volume)



0
npy: 0.009683609008789062s, f0: 0.0006194114685058594s, infer: 0.059151411056518555s


In [12]:
from scipy.io.wavfile import write as write_wav

# save audio to disk
write_wav(f"test.wav", out_sr, output_audio) 

# play text in notebook
Audio(output_audio, rate=out_sr)

In [10]:
json_data = '''
[
    {
        "title": "菩萨蛮 其一",
        "paragraphs": [
            "小山重叠金明灭，鬓云欲度香腮雪。",
            "懒起画蛾眉，弄妆梳洗迟。",
            "照花前后镜，花面交 相映。",
            "新帖绣罗襦，双双金鹧鸪。"
        ],
        "author": "温庭筠",
        "rhythmic": "菩萨蛮",
        "notes": [
            "1.小山--写女子的隔夜残妆。小山：女子画眉的式样之一。小山重叠：眉晕褪色。金：额黄，在额上涂黄色。金明灭：褪色的额黄明暗不匀。",
            "2.鬓云欲度--鬓发撩乱如云，低垂下来。香腮雪：洁白如雪的香腮。",
            "3.照花--对镜簪花。用前镜、后镜对照以瞻顾后影。",
            "4.双双--罗襦上用金线绣的成双的鹧鸪鸟。反衬自身孤独。"
        ]
    }
]
'''

In [18]:
import json
import pyaudio
import numpy as np
import asyncio

#conda install -c anaconda portaudio    


async def play_audio(audio_data, rate):
    loop = asyncio.get_event_loop()
    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        nonlocal audio_data
        data = audio_data[:frame_count]
        audio_data = audio_data[frame_count:]
        return (data.tobytes(), pyaudio.paContinue)

    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=rate,
                    output=True,
                    stream_callback=callback)

    stream.start_stream()
    while stream.is_active():
        await asyncio.sleep(0.1)

    stream.stop_stream()
    stream.close()
    p.terminate()


In [19]:
import time


async def text_to_speech(data):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=22050,
                    output=True)
    for item in data:
        
        timestamp = int(time.time())
        for key, value in item.items():
            
            if isinstance(value, str): 
                out_sr, output_audio = await edge_tts_vc_func(rvc,value,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                        feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                           rate=rate, volume=volume)
                 
                # save audio to disk
                write_wav(f"wav/{timestamp}_{key}.wav", out_sr, output_audio) 
                await play_audio(output_audio,out_sr)
            elif isinstance(value, list):
                for i, para in enumerate(value): 
                    out_sr, output_audio = await edge_tts_vc_func(rvc,para,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                        feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                           rate=rate, volume=volume)
                    
                    # save audio to disk
                    write_wav(f"wav/{timestamp}_{key}_{i}.wav", out_sr, output_audio) 
                    await play_audio(output_audio,out_sr)

In [28]:

import os

'''
获取输入文件夹内的所有wav文件，并返回文件名全称列表
'''
def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        print(files)
        for file in files:
            if os.path.splitext(file)[1] == '.json':
                filename=os.path.join(root, file)
                L.append(filename)
    return L 

def read_text_file(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return None
    except Exception as e:
        print("An error occurred while reading the file:", e)
        return None

    
json_dir="/media/checkpoint/chinese-poetry/曹操诗集/"#文件夹
json_texts=file_name(json_dir)#获取文件夹内的所有文件
print(json_texts)
for text in json_texts:
    text_content = read_text_file(text)
    data = json.loads(text_content)
    await text_to_speech(data)

    

['README.md', 'caocao.json']
['/media/checkpoint/chinese-poetry/曹操诗集/caocao.json']


ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

npy: 0.024892330169677734s, f0: 1.5089645385742188s, infer: 0.12712907791137695s


ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

npy: 0.025900602340698242s, f0: 1.9332213401794434s, infer: 0.1336216926574707s


...

npy: 0.010014533996582031s, f0: 2.4657938480377197s, infer: 0.07212281227111816s


ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

npy: 0.010982275009155273s, f0: 2.2076070308685303s, infer: 0.06216740608215332s


ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

npy: 0.026071548461914062s, f0: 1.9284298419952393s, infer: 0.1286485195159912s
